# HOMEWORK 6: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming)

We will focus only on the Object Classification task for this homework.

In this homework, you are asked compare different text classification models in terms of accuracy and inference time.

You will need to build 3 different models.

1. A model based on tf-idf
2. A model based on MUSE
3. A model based on wangchanBERTa

**You will be ask to submit 3 different files (.pdf from .ipynb) that does the 3 different models. Finally, answer the accuracy and runtime numbers in MCV.**

This homework is quite free form, and your answer may vary. We hope that the processing during the course of this assignment will make you think more about the design choices in text classification.

In [1]:
# !wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
# !pip install pythainlp

## Import Libs

In [2]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score

#My import 
np.random.seed(42)
from sklearn.model_selection import train_test_split


## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [3]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [4]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 0.1:
You will have to remove unwanted label duplications as well as duplications in text inputs.
Also, you will have to trim out unwanted whitespaces from the text inputs.
This shouldn't be too hard, as you have already seen it in the demo.



In [5]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [6]:
# TODO.1: Data cleaning

# select only the columns that we need
data_df = data_df[['Sentence Utterance', 'Object']]
data_df.columns = ['input', 'raw_label']

# remove duplicate labels
data_df['clean_label'] = data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)

# ! input to lower case (helps with OOV token)
data_df['input'] = data_df['input'].str.lower()

# remove duplicate input rows
data_df = data_df.drop_duplicates("input", keep="first")
display(data_df.describe())

,input,clean_label
count,13385,13385
unique,13385,26
top,สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ,service
freq,1,2108


In [7]:
# map label to number and convert to numpy "data"
data = data_df.to_numpy()
unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

# convert label to number
data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])

In [8]:
# Input string cleaning
def strip_str(string):
    return string.strip()
     
# Trim of extra begining and trailing whitespace in the string
data[:,0] = np.vectorize(strip_str)(data[:,0])

### Keep in mind class is imbalance type shi

Split data into train, valdation, and test sets (normally the ratio will be 80:10:10 , respectively). We recommend to use train_test_spilt from scikit-learn to split the data into train, validation, test set.

In addition, it should split the data that distribution of the labels in train, validation, test set are similar. There is **stratify** option to handle this issue.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Make sure the same data splitting is used for all models.

In [9]:
X = data[:,0]
y = data[:,1]

X_class_24, y_class_24 = X[y == 24], y[y == 24] #handle class 24 with only 4 samples
X, y = X[y != 24], y[y != 24] # remove class 24 from the data to add it later

random_idx = np.random.choice(len(X_class_24), 2, replace=False)

X_class_24_val, X_class_24_test = X_class_24[random_idx[0]], X_class_24[random_idx[1]]
y_class_24_val, y_class_24_test = y_class_24[random_idx[0]], y_class_24[random_idx[1]]

X_class_24_train = np.delete(X_class_24, random_idx)
y_class_24_train = np.delete(y_class_24, random_idx)

# train val test split 80:10:10
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42)

# add class 24 to the train, val, test set
X_train, y_train = np.append(X_train, X_class_24_train), np.append(y_train, y_class_24_train)
X_val, y_val = np.append(X_val, X_class_24_val), np.append(y_val, y_class_24_val)
X_test, y_test = np.append(X_test, X_class_24_test), np.append(y_test, y_class_24_test)

#check label distribution
# display(np.unique(y_train, return_counts=True))

In [10]:
def create_dataset_dict(dataset, data_split, split_name, keep_ws=False):
    for input_str, label in data_split:
        dataset[split_name]["input"].append(input_str)
        dataset[split_name]["label"].append(label)

dataset = { "train": {"input": [], "label": []},
            "val": {"input": [], "label": []},
            "test": {"input": [], "label": []} }

create_dataset_dict(dataset, zip(X_train, y_train), "train")
create_dataset_dict(dataset, zip(X_val, y_val), "val")
create_dataset_dict(dataset, zip(X_test, y_test), "test")

print(dataset["train"]["input"][:5])
print(dataset["train"]["label"][:5])

['อยากจะสอบถามทรูมูฟ เอช 3g ที่ตำบลบ้านกลาง มีสัญญาณหรือเปล่าค่ะ', 'เติมเงินแล้วไม่ได้', 'จะเติมเงินต้องทำยังไง', 'ถ้าจะซื้อ เดอะ นิวไอแพด ใช้บัตรอะไรรูดได้บ้าง', 'เมื่อคืนนี้ผมโทรมาแจ้งเรื่องโทรศัพท์หาไม่เจอ และระงับไว้ ตอนนี้เจอ อยู่ใต้เบาะรถ ต้องการเปิดสัญญาณการใช้งาน']
[4, 0, 0, 16, 5]


# Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:** -> **"newmm"** . fast and efficient (Good enough acc on BEST test benchmark word level tokenization)

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:** I will not use stop word. but instead use the TfidfVectorizer()'s 'max_df' to cut out to frequent words.

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:**

- Number of OOV words in validation set: 190
- Number of OOV words in test set: 185
- OOV ratio in validation set: 0.05255878284923928
- OOV ratio in test set: 0.051175656984785614

In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

print(set(thai_stopwords()))

{'ระยะ', 'นาน', 'สิ้นกาลนาน', 'ที่แห่งนั้น', 'ทว่า', 'นั่นเอง', 'อันที่', 'หนึ่ง', 'การ', 'ภายนอก', 'แห่งไหน', 'เปลี่ยนแปลง', 'เฉย', 'ทั้งนั้นเพราะ', 'ที่ไหน', 'ถึงแม้', 'นอกเหนือจาก', 'สิ่งใด', 'สุดๆ', 'รวม', 'ตลอดมา', 'มั้ยนะ', 'ขณะเดียวกัน', 'นั้นไว', 'เช่นนั้นเอง', 'ทุกคน', 'จู่ๆ', 'จง', 'ไฉน', 'เผื่อจะ', 'เสียนั่นเอง', 'บอกว่า', 'อดีต', 'ขอ', 'พึ่ง', 'แล้วเสร็จ', 'รวมด้วย', 'เหลือ', 'ช่วงที่', 'มีแต่', 'ปฏิบัติ', 'นาย', 'พอกัน', 'ใหม่', 'เมื่อครั้งก่อน', 'นี้แหล่', 'ช่วงท้าย', 'รวดเร็ว', 'หมด', 'จวบจน', 'วันใด', 'เป็นต้นไป', 'คล้ายว่า', 'ทุกหน', 'เช่นเมื่อ', 'ใคร่', 'เกือบจะ', 'หากว่า', 'คล้ายกัน', 'เกือบๆ', 'กลับ', 'ขณะใด', 'เมื่อเย็น', 'ผู้ใด', 'แต่ก็', 'ถึงแม้ว่า', 'ละ', 'เพียงแต่', 'ช่วงนี้', 'อย่างไร', 'เป็นการ', 'เสียยิ่ง', 'พวกฉัน', 'นี่', 'จัดหา', 'ดั่งเก่า', 'ก็ต่อเมื่อ', 'จริง', 'พอสม', 'กระผม', 'อัน', 'เพราะว่า', 'เป็นเพียง', 'นั่นเป็น', 'มุ่ง', 'กลุ่มๆ', 'เสีย', 'เป็นอาทิ', 'กำลังจะ', 'ที่ใด', 'ก็ตามแต่', 'รับ', 'เชื่อถือ', 'ด้วย', 'อันๆ', 'ขึ้น', 'น้อย', 'ภาคฯ', 'ด้วย

In [12]:
pythainlp_tokenizer = lambda x: word_tokenize(x, engine="newmm", keep_whitespace=False)

# Extract tokenized text and labels
train_texts, train_labels = dataset["train"]["input"], dataset["train"]["label"]
val_texts, val_labels = dataset["val"]["input"], dataset["val"]["label"]
test_texts, test_labels = dataset["test"]["input"], dataset["test"]["label"]

# Initialize and fit TF-IDF Vectorizer on training data
vectorizer = TfidfVectorizer(tokenizer=pythainlp_tokenizer, max_df=0.7, min_df=1)
X_train_tfidf = vectorizer.fit_transform(train_texts)  # Fit and transform training data

X_val_tfidf = vectorizer.transform(val_texts)  # Transform validation data
X_test_tfidf = vectorizer.transform(test_texts)  # Transform test data

# Print shape of TF-IDF matrices
print("Train shape:", X_train_tfidf.shape)
print("Val shape:", X_val_tfidf.shape)
print("Test shape:", X_test_tfidf.shape)

# vectorizer get feature names
feature_names = vectorizer.get_feature_names_out()
print("Number of features:", len(feature_names))

# Print first 5 samples of processed train data
print("Sample TF-IDF Features (First 5 training samples):")
print(X_train_tfidf[0])

/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Train shape: (10706, 3615)
Val shape: (1339, 3615)
Test shape: (1340, 3615)
Number of features: 3615
Sample TF-IDF Features (First 5 training samples):
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15 stored elements and shape (1, 3615)>
  Coords	Values
  (0, 2582)	0.15335071962729047
  (0, 1065)	0.10887956336518148
  (0, 2376)	0.12784073325007023
  (0, 1445)	0.2091753985526012
  (0, 3216)	0.24593021912326413
  (0, 177)	0.24833353650083592
  (0, 434)	0.27233973099167436
  (0, 1499)	0.1642673698623802
  (0, 1349)	0.4665220808838139
  (0, 1714)	0.2926490948729209
  (0, 761)	0.48161481885581436
  (0, 1974)	0.17203910879251177
  (0, 2390)	0.19466884830085984
  (0, 2526)	0.257246993019674
  (0, 1017)	0.11098951226265677


In [13]:
# check for oov in test and val set
word_dict = vectorizer.get_feature_names_out()

oov_set = set()

def get_oov_words(texts):
    oov_set = set()
    for text in texts:
        for word in pythainlp_tokenizer(text):
            if word not in word_dict:
                oov_set.add(word)
    return oov_set

oov_val = get_oov_words(val_texts)
oov_test = get_oov_words(test_texts)

print("Number of OOV words in validation set:", len(oov_val))
print("Number of OOV words in test set:", len(oov_test))
print("OOV ratio in validation set:", len(oov_val) / len(word_dict))
print("OOV ratio in test set:", len(oov_test) / len(word_dict))

Number of OOV words in validation set: 190
Number of OOV words in test set: 185
OOV ratio in validation set: 0.05255878284923928
OOV ratio in test set: 0.051175656984785614


## Define a LogisticRegression model with tf-idf as feature.

In [14]:
#import from sklearn that automatically select the best hyperparameter
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define the hyperparameter grid
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear"]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV( estimator=LogisticRegression(max_iter=1000, class_weight="balanced"), 
                            param_grid=param_grid, 
                            cv=5, 
                            n_jobs=-1, 
                            verbose=2)

In [15]:
# Fit the GridSearchCV object on the training data
grid_search.fit(X_train_tfidf, train_labels)

# get the best model
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV] END ...............C=0.01, penalty=l1, solver=liblinear; total time=   0.2s
[CV] END ...............C=0.01, penalty=l1, solver=liblinear; total time=   0.2s
[CV] END ...............C=0.01, penalty=l1, solver=liblinear; total time=   0.3s
[CV] END ...............C=0.01, penalty=l1, solver=liblinear; total time=   0.3s
[CV] END ...............C=0.01, penalty=l2, solver=liblinear; total time=   0.3s
[CV] END ...............C=0.01, penalty=l2, solver=liblinear; total time=   0.4s
[CV] END ...............C=0.01, penalty=l1, solver=liblinear; total time=   0.3s
[CV] END ...............C=0.01, penalty=l2, solver=liblinear; total time=   0.4s
[CV] END ................C=0.1, penalty=l1, solver=liblinear; total time=   0.5s
[CV] END ................C=0.1, penalty=l1, solver=liblinear; total time=   0.5s
[CV] END ................C=0.1, penalty=l2, solver=liblinear; total time=   0.6s
[CV] END ...............C=0.01, penalty=l2, solver=liblinear; total time=   0.6s
[CV] END ...............C=0.

/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END .................C=10, penalty=l1, solver=liblinear; total time=   3.9s
[CV] END ................C=100, penalty=l1, solver=liblinear; total time=   4.1s
[CV] END ................C=100, penalty=l1, solver=liblinear; total time=   4.4s
[CV] END ................C=100, penalty=l1, solver=liblinear; total time=   4.3s
[CV] END ................C=100, penalty=l1, solver=liblinear; total time=   4.4s
[CV] END ................C=100, penalty=l1, solver=liblinear; total time=   4.1s


/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END .................C=10, penalty=l1, solver=liblinear; total time=   8.5s


/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END .................C=10, penalty=l1, solver=liblinear; total time=  11.2s


In [16]:
best_model

LogisticRegression(C=10, class_weight='balanced', max_iter=1000,
                   solver='liblinear')

# Evaluation

In [ ]:
# Evaluate the best model on the validation data
val_preds = best_model.predict(X_val_tfidf)
val_acc = accuracy_score(val_labels, val_preds)
print("Validation Accuracy:", val_acc)

test_preds = best_model.predict(X_test_tfidf)
test_acc = accuracy_score(test_labels, test_preds)
print("Test Accuracy:", test_acc)


# Generate classification report on test data
print(classification_report(test_labels, test_preds, target_names=num_2_label_map.values()))

Validation Accuracy: 0.7326362957430919
Test Accuracy: 0.7231343283582089
                 precision    recall  f1-score   support

        payment       0.64      0.73      0.68        64
        package       0.76      0.69      0.72       180
        suspend       0.77      0.81      0.79        73
       internet       0.75      0.77      0.76       179
   phone_issues       0.63      0.72      0.67        58
        service       0.85      0.70      0.77       211
    nontruemove       0.42      0.64      0.51        25
        balance       0.85      0.74      0.80       149
         detail       0.44      0.52      0.47        33
           bill       0.67      0.83      0.74        54
         credit       0.71      0.88      0.79        17
      promotion       0.72      0.68      0.70       115
 mobile_setting       0.48      0.50      0.49        28
       iservice       1.00      1.00      1.00         2
        roaming       0.81      0.88      0.85        25
      truemon

/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jaf/anaconda3/envs/colab-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

# Model 2 MUSE

Build a simple logistic regression model using features from the MUSE model.

Which MUSE model will you use? Why?

**Ans:**

MUSE is typically used with tensorflow. However, there are some pytorch conversions made by some people.

https://huggingface.co/sentence-transformers/use-cmlm-multilingual
https://huggingface.co/dayyass/universal-sentence-encoder-multilingual-large-3-pytorch

# Model 3 WangchanBERTa

We ask you to train a WangchanBERTa-based model.

We recommend you use the thaixtransformers fork (which we used in the PoS homework).
https://github.com/PyThaiNLP/thaixtransformers

The structure of the code will be very similar to the PoS homework. You will also find the huggingface [tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) useful. Or you can also add a softmax layer by yourself just like in the previous homework.

Which WangchanBERTa model will you use? Why? (Don't forget to clean your text accordingly).

**Ans:**


After you

# Comparison

After you have completed the 3 models, compare the accuracy, ease of implementation, and inference speed (from cleaning, tokenization, till model compute) between the three models in mycourseville.